In [8]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()

# Conjunto de entrada
X = iris.data[:,2:] # Largura e comprimento da petala
y = iris.target

# Max_depth = profundidade maxima(Hiperparametro)
tree_clf = DecisionTreeClassifier(max_depth=2,random_state=42)
tree_clf.fit(X,y)


array([1])

In [7]:
tree_clf.predict_proba([[5,1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [9]:
tree_clf.predict([[5,1.5]])

array([1])

In [69]:
import pandas as pd

df = pd.read_excel('Play.xlsx')

# Converte todas as colunas para tipo categórico
df_categorical = df.astype('category')

# Transforma cada coluna em códigos numéricos
df_encoded = df_categorical.apply(lambda x: x.cat.codes)


X = df_encoded.iloc[:,[1,2,3,4]]
y = df_encoded.iloc[:,[5]] 

tree_clf = DecisionTreeClassifier(max_depth=5,random_state=42)
tree_clf.fit(X,y)

print(df_encoded)


    DIA  TEMPO  TEMPERATURA  UMIDADE  VENTO  JOGA
0     0      1            2        0      1     0
1     6      1            2        0      0     0
2     7      2            2        0      1     1
3     8      0            1        0      1     1
4     9      0            0        1      1     1
5    10      0            0        1      0     0
6    11      2            0        1      0     1
7    12      1            1        0      1     0
8    13      1            0        1      1     1
9     1      0            1        1      1     0
10    2      1            1        1      0     0
11    3      2            1        0      0     0
12    4      2            2        1      1     0
13    5      0            1        0      0     1


In [73]:
tree_clf.predict_proba([[6,1,2,0]])


/home/michael/miniconda3/envs/ml/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([[1., 0.]])

In [74]:
tree_clf.predict([[6,1,2,0]])


/home/michael/miniconda3/envs/ml/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([0], dtype=int8)

### Exercicio 3 
classificar o conjunto de dados "moons"(make moons)
a) Dividir entre treino e teste
b) Encontrar bons valores de hiperparâmetros para o classificados, utilizando a classe GridSearchCV

In [104]:
from sklearn.datasets import make_moons
from sklearn.model_selection import GridSearchCV

# GERAR DADOS
X,y = make_moons(n_samples=10000,random_state=42)

#  hiperparemetros para teste
param_grid = {
    'max_depth': [1, 2, 3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 4, 6]
}

# Treinar modelo
tree_clf = DecisionTreeClassifier(random_state=42)

grid_search = GridSearchCV(tree_clf,param_grid, scoring='accuracy')
grid_search.fit(X, y)

print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor score médio de validação:", grid_search.best_score_)

Melhores parâmetros: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}
Melhor score médio de validação: 0.9994


### Exercício 4
Criar uma floresta, com base no exercício anterior:

a)Gerar mil subconjunto do conjunto de treinamento, cada um contendo 100 instâncias selecionadas aleatoriamente. (Dica: usar a classe ShuffleSplit da Scikit-Learn.)

b)Treinar uma árvore de decisão em cada subconjunto, usando os melhores valores de hiperparâmetros encontrados no exercício anterior.

c)Para cada instância do conjunto de testes, gere as predições das mil árvores de decisão e mantenha somente as predições mais frequentes (Dica, usar a função mode() do SciPy). Essa abordagem fornece as predições dos votos majoritários em relação ao conjunto de testes.

d)Avalie essas predições no conjunto de testes: você deve obter uma acurácia um pouco maior do que a do seu primeiro modelo (cerca de 0,5 a 1,5% maior).

In [105]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from scipy.stats import mode
from sklearn.metrics import accuracy_score
import numpy as np


ss = ShuffleSplit(n_splits=1000, train_size=100, random_state=42)
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

floresta = []

for train_indices, _ in ss.split(X_train_full):
    X_sub = X_train_full[train_indices]
    y_sub = y_train_full[train_indices]
    
    tree_clf = DecisionTreeClassifier(max_depth=5,min_samples_leaf=4,min_samples_split=2, random_state=42)
    tree_clf.fit(X_sub, y_sub)
    
    floresta.append(tree_clf)

all_predictions = np.zeros((len(floresta), len(X_test)))

for idx, tree in enumerate(floresta):
    all_predictions[idx] = tree.predict(X_test)

majority_votes, _ = mode(all_predictions, axis=0, keepdims=True)
y_pred_forest = majority_votes.flatten()


accuracy = accuracy_score(y_test, y_pred_forest)
print("Acurácia da floresta (votação majoritária):", accuracy)

Acurácia da floresta (votação majoritária): 0.9915
